Basato su EVALITA 2020 HaSpeeDe (http://www.di.unito.it/~tutreeb/haspeede-evalita20/index.html)

Sviluppare un classificatore basato su SVM lineari che prende in input una rappresentazione del testo basata su n-grammi di caratteri, parole e part-of-speech. Riportare i seguenti risultati:
- Testare diverse rappresentazioni del testo che variano rispetto alla lunghezza degli ngrammi utilizzati e/o rispetto al tipo di informazione utilizzata all’interno degli ngrammi (forme, lemmi, caratteri, part-of-speech) e valutare i diversi sistemi con un processo di 5-fold cross validation condotto sul training set;
- Valutazione sul test set ufficiale del miglior sistema rispetto ai risultati ottenuti con il processo di 5-fold cross validation del punto sopra.

I file delle annotazioni linguistiche sono già stati estratti tramite Profiling-UD.

In [1]:
import os
#path alla cartella in cui sono presenti i file .conllu per il training set da 6383 elementi
conllu_dir = 'profiling_output/11925/'

In [2]:
#inizio col definire le classi document, sentence e token 
class Document: #un documento è una lista di frasi
    #inizializza il percorso, chiama parse_doc_info per estrarre info dal nome del file. poi inizializza due variabili per estrarre informazioni dal documento: lista di frasi e features
    def __init__(self, document_path):
        self.document_path = document_path
        self._parse_doc_info(document_path)
        self.sentences = []
        self.features = None
    #estrae id e hs dal nome del file
    def _parse_doc_info(self, document_path):
        document_path = document_path.split('/')[-1]
        document_info = document_path.split('.')[0]
        #splitto la stringa del nome del file in una lista che ha come primo elemento (prima del #) l'id e come secondo (dopo il #) l'hs (0 se non contiene hs, 1 se lo contiene)
        document_info = document_info.split('#')
        self.id = document_info[0]
        self.hs = document_info[1]
    #aggiunge una frase alla lista delle frasi         
    def add_sentence(self, sentences):
        self.sentences.append(sentences)
    #restituisce il numero di token totali del documento   
    def get_num_tokens(self):
        num_words = 0
        for sentence in self.sentences:
            num_words = num_words + sentence.get_num_tokens()
        return num_words
    #restituisce il numero totale di caratteri del documento
    def get_num_chars(self):
        num_chars = 0
        for sentence in self.sentences:
            sentence_char_len = sentence.get_num_chars()
            num_chars = num_chars + sentence_char_len
        return num_chars
        
class Sentence: #una frase è una lista di token
    #inizializza la lista di token
    def __init__(self):
        self.tokens = []
    #aggiunge un token alla lista di token 
    def add_token(self, token):
        self.tokens.append(token)
    #restituisce la lista delle parole token (token presi una volta sola, non ripetuti) nella frase
    def get_words(self):
        return [token.word for token in self.tokens]
    #restituisce la lista dei lemmi nella frase
    def get_lemmas(self):
        return [token.lemma for token in self.tokens]
    #restituisce la lista dei POS nella frase
    def get_pos(self):
        return [token.pos for token in self.tokens]
    #restituisce il numero di token nella frase
    def get_num_tokens(self):
        return len(self.tokens)
    #restituisce il numero totale di caratteri nella frase (spazi inclusi, come specificato)
    def get_num_chars(self):
        num_chars = 0
        for token in self.tokens:
            num_chars = num_chars + token.get_num_chars()
        num_chars = num_chars + self.get_num_tokens() - 1 # contiamo anche gli spazi
        return num_chars
    #restituisce rappresentazione stringa della frase combinando tutti i token, separati da spazi
    def __str__(self):
        return ' '.join([token.word for token in self.tokens])

class Token: #è un singolo token
    #inizializza la parola token, il lemma e il POS
    def __init__(self, word, lemma, pos):
        self.word = word
        self.lemma = lemma
        self.pos = pos
    #restituisce il numero di caratteri della parola token
    def get_num_chars(self):
        return len(self.word)

In [3]:
#carica un documento leggendone frasi e token
def load_document_sentences(document):
    sentence = Sentence()
    for line in open(document.document_path, 'r'): #cicla sulle righe del file
        if line[0].isdigit():  #controlla se la riga inizia con un numero perché vorrebbe dire che siamo davanti a un token nel caso di un file .conllu
            splitted_line = line.strip().split('\t')
            if '-' not in splitted_line[0]:  #se l'id della parola non contiene un trattino (tutti gli id non dovrebbero contenere trattini)
                token = Token(splitted_line[1], splitted_line[2], splitted_line[3])
                #aggiunge il token alla frase
                sentence.add_token(token)
        if line == '\n':  #se la riga è vuota significa che la frase è finita
            #aggiunge la frase al documento
            document.add_sentence(sentence)
            #inizializza una nuova frase
            sentence = Sentence()

In [4]:
#legge tutti i file di una cartella
all_documents = [] #inizializza la lista dei documenti, per ora vuota
for file_name in os.listdir(conllu_dir): #itera i file nella cartella
    file_path = os.path.join(conllu_dir, file_name)
    #per ogni file, crea un oggetto Document
    document = Document(file_path)
    #utilizza la funzione load_document_sentences per caricare il documento leggendone frasi e token
    load_document_sentences(document)
    #aggiunge il documento alla lista dei documenti
    all_documents.append(document)

In [5]:
#funzione per estrarre n-grammi da una frase: el può essere "word" per n-grammi di parole, "lemma" per n-grammi di lemmi o "pos" per n-grammi di pos
def extract_word_ngrams_from_sentence(word_ngrams, sentence, el, n): #word_ngrams è un dizionario di n-grammi (n-grammi chiavi, numero di volte in cui appaiono valori)
    #crea una lista con tutte le parole
    if el == 'word':
        all_words = sentence.get_words()
    elif el == 'lemma':
        all_words = sentence.get_lemmas()
    elif el == 'pos':
        all_words = sentence.get_pos()
    else:
        raise Exception(f'Invalid element {el}')
    #all_words è la lista delle parole, dei lemmi o dei pos

    #scorre la lista delle parole ed estrae gli n-grammi
    for i in range(0, len(all_words) - n + 1): #-n+1 serve per non uscire dal vettore
        ngram_words = all_words[i: i + n]
        ngram = f'{el.upper()}_{n}_' + '_'.join(ngram_words) #crea l'n-gramma come stringa
        if ngram not in word_ngrams: #aggiunge l'n-gramma al dizionario, o aumenta di 1 il suo valore se già presente
            word_ngrams[ngram] = 1
        else:
            word_ngrams[ngram] += 1
    #ritorna il dizionario
    return word_ngrams

In [6]:
#funzione per estrarre n-grammi di caratteri da una frase
def extract_char_ngrams_from_sentence(char_ngrams, sentence, n):
    #crea una lista con tutte le parole
    all_words = sentence.get_words()
    #crea una stringa che contenga tutte le parole separate tra spazi perchè vogliamo scorrere i caratteri
    all_words = ' '.join(all_words)

    #scorre la stringa ed estrae gli n-grammi di caratteri
    for i in range(0, len(all_words) - n + 1):
        ngram_chars = all_words[i:i + n]
        ngram = f'CHAR_{n}_' + ngram_chars #crea l'n-gramma come stringa
        if ngram not in char_ngrams: #aggiunge l'n-gramma al dizionario, o aumenta di 1 il suo valore se già presente
            char_ngrams[ngram] = 1
        else:
            char_ngrams[ngram] += 1
    #ritorna il dizionario
    return char_ngrams

In [7]:
#per ogni documento, estrae n-grammi di parole e/o di caratteri e li memorizza come attributo features, normalizzandoli
def normalize_ngrams(ngrams_dict, doc_len):
    for ngram in ngrams_dict:
        ngrams_dict[ngram] = ngrams_dict[ngram] / float(doc_len)

#ngram_type può essere "word", "lemma", "pos" o "char"; ngram_number può essere qualsiasi numero da 1 in su
def extract_documents_ngrams_normalized(all_documents, ngram_type, ngram_number):
    #controllo che ngram_number sia 1 o più
    if ngram_number < 1:
        raise Exception(f'Invalid element {ngram_number}')
    
    for document in all_documents:
        
        ngrams = dict()
        
        for sentence in document.sentences:
            if ngram_type in ["word", "lemma", "pos"]:
                extract_word_ngrams_from_sentence(ngrams, sentence, ngram_type, ngram_number)
                num_words = document.get_num_tokens()
                normalize_ngrams(ngrams, num_words)
            elif ngram_type == "char":
                extract_char_ngrams_from_sentence(ngrams, sentence, ngram_number)
                num_chars = document.get_num_chars()
                normalize_ngrams(ngrams, num_chars)

        document.features = ngrams
#alla fine, ogni documento della lista all_documents avrà un attributo features contenente un dizionario di n-grammi normalizzati.

### Rappresentazione del testo basata su bigrammi di parole

In [8]:
extract_documents_ngrams_normalized(all_documents, "word", 2)

In [9]:
features = [document.features for document in all_documents]
labels = [document.hs for document in all_documents]

In [10]:
#filtro le features rimuovendo quelle che compaiono meno di "min_occurencies"

def get_num_features(features_dict):
    all_features = set()
    for document_feats in features_dict:
        all_features.update(list(document_feats.keys()))
    return len(all_features)

def filter_features(train_features_dict, min_occurrences):
    #contiamo ogni feature in quanti dcoumenti diversi compare
    features_counter = dict()
    for document_features_dict in train_features_dict:
        for feature in document_features_dict:
            if feature in features_counter:
                features_counter[feature] += 1
            else:
                features_counter[feature] = 1

    #per ogni documento, togliamo le features che compaiono meno di "min_occurrences"
    for document_features_dict in train_features_dict:
        document_features = list(document_features_dict.keys())
        for feature in document_features:
            if features_counter[feature] < min_occurrences:
                document_features_dict.pop(feature)

    return features

print(f'Numero features iniziali: {get_num_features(features)}')

#applico il filtro mettendo min_occurencies a 5
features = filter_features(features, 5)

print(f'Numero features dopo il filtro: {get_num_features(features)}')

Numero features iniziali: 98227
Numero features dopo il filtro: 4036


In [11]:
#importo le librerie per fare k-fold cross validation
import numpy as np
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction import DictVectorizer

#inizializzo il vettorizzatore per ottenere poi delle matrici numeriche
vectorizer = DictVectorizer()

#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(features))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [features[idx] for idx in train_ids]
    fold_y_train = np.asarray([labels[idx] for idx in train_ids])
    fold_X_test = [features[idx] for idx in test_ids]
    fold_y_test = np.asarray([labels[idx] for idx in test_ids])
    
    #uso dict_vectorizer per trasformare fold_X_train e fold_X_test in matrici numeriche (verranno sparse)
    fold_X_train = vectorizer.fit_transform(fold_X_train)
    fold_X_test = vectorizer.transform(fold_X_test)
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.618421052631579, baseline: 0.5986842105263158
Accuracy fold 2: 0.6089181286549707, baseline: 0.591374269005848
Accuracy fold 3: 0.6254572055596196, baseline: 0.60424286759327
Accuracy fold 4: 0.6057059253840527, baseline: 0.5874177029992684
Accuracy fold 5: 0.6100950987564009, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.61      0.99      0.75      4071
           1       0.78      0.06      0.12      2766

    accuracy                           0.61      6837
   macro avg       0.69      0.53      0.44      6837
weighted avg       0.68      0.61      0.50      6837



Riapplico adesso la stessa procedura appena utilizza per le rappresentazioni basate su bigrammi di lemmi, POS e caratteri.

### Rappresentazione del testo basata su bigrammi di lemmi

In [12]:
extract_documents_ngrams_normalized(all_documents, "lemma", 2)

In [13]:
features = [document.features for document in all_documents]
labels = [document.hs for document in all_documents]

In [14]:
print(f'Numero features iniziali: {get_num_features(features)}')

#applico il filtro mettendo min_occurencies a 5
features = filter_features(features, 5)

print(f'Numero features dopo il filtro: {get_num_features(features)}')

Numero features iniziali: 82536
Numero features dopo il filtro: 4504


In [15]:
#inizializzo il vettorizzatore per ottenere poi delle matrici numeriche
vectorizer = DictVectorizer()

#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(features))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [features[idx] for idx in train_ids]
    fold_y_train = np.asarray([labels[idx] for idx in train_ids])
    fold_X_test = [features[idx] for idx in test_ids]
    fold_y_test = np.asarray([labels[idx] for idx in test_ids])
    
    #uso dict_vectorizer per trasformare fold_X_train e fold_X_test in matrici numeriche (verranno sparse)
    fold_X_train = vectorizer.fit_transform(fold_X_train)
    fold_X_test = vectorizer.transform(fold_X_test)
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.6264619883040936, baseline: 0.5986842105263158
Accuracy fold 2: 0.6220760233918129, baseline: 0.591374269005848
Accuracy fold 3: 0.6313094367227505, baseline: 0.60424286759327
Accuracy fold 4: 0.6166788588149232, baseline: 0.5874177029992684
Accuracy fold 5: 0.6217995610826628, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.62      0.98      0.76      4071
           1       0.75      0.11      0.18      2766

    accuracy                           0.62      6837
   macro avg       0.68      0.54      0.47      6837
weighted avg       0.67      0.62      0.52      6837



### Rappresentazione del testo basata su bigrammi di POS

In [16]:
extract_documents_ngrams_normalized(all_documents, "pos", 2)

In [17]:
features = [document.features for document in all_documents]
labels = [document.hs for document in all_documents]

In [18]:
print(f'Numero features iniziali: {get_num_features(features)}')

#applico il filtro mettendo min_occurencies a 5
features = filter_features(features, 5)

print(f'Numero features dopo il filtro: {get_num_features(features)}')

Numero features iniziali: 246
Numero features dopo il filtro: 229


In [19]:
#inizializzo il vettorizzatore per ottenere poi delle matrici numeriche
vectorizer = DictVectorizer()

#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(features))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [features[idx] for idx in train_ids]
    fold_y_train = np.asarray([labels[idx] for idx in train_ids])
    fold_X_test = [features[idx] for idx in test_ids]
    fold_y_test = np.asarray([labels[idx] for idx in test_ids])
    
    #uso dict_vectorizer per trasformare fold_X_train e fold_X_test in matrici numeriche (verranno sparse)
    fold_X_train = vectorizer.fit_transform(fold_X_train)
    fold_X_test = vectorizer.transform(fold_X_test)
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.6352339181286549, baseline: 0.5986842105263158
Accuracy fold 2: 0.6308479532163743, baseline: 0.591374269005848
Accuracy fold 3: 0.6269202633504023, baseline: 0.60424286759327
Accuracy fold 4: 0.6188734455010972, baseline: 0.5874177029992684
Accuracy fold 5: 0.6269202633504023, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.63      0.89      0.74      4071
           1       0.60      0.25      0.35      2766

    accuracy                           0.63      6837
   macro avg       0.62      0.57      0.54      6837
weighted avg       0.62      0.63      0.58      6837



### Rappresentazione del testo basata su bigrammi di caratteri

In [20]:
extract_documents_ngrams_normalized(all_documents, "char", 2)

In [21]:
features = [document.features for document in all_documents]
labels = [document.hs for document in all_documents]

In [22]:
print(f'Numero features iniziali: {get_num_features(features)}')

#applico il filtro mettendo min_occurencies a 5
features = filter_features(features, 5)

print(f'Numero features dopo il filtro: {get_num_features(features)}')

Numero features iniziali: 3390
Numero features dopo il filtro: 1441


In [23]:
#inizializzo il vettorizzatore per ottenere poi delle matrici numeriche
vectorizer = DictVectorizer()

#inizializzo k-fold
splitter = KFold(n_splits=5, random_state=42, shuffle=True) #faccio 5-folds, utilizzo random_state 42 per la riproducibilità, e shuffle mescola i dati prima di distribuirli nei folds
folds = list(splitter.split(features))

#stampo le dimensioni dei folds per assicurarmi che tutto abbia funzionato correttamente
for i in range(len(folds)):
    print(len(folds[i][0]), len(folds[i][1]))
    
all_y_true = [] #etichette vere
all_y_pred = [] #etichette predette

for i in range(len(folds)): #itero sui folds generati prima
    #prendo i dati di training e di test per l'attuale fold
    train_ids = folds[i][0]
    test_ids = folds[i][1]
    
    #creo training set e test set per l'attuale fold estrando le righe corrispondenti agli indici specificati in train_ids o test_ids
    fold_X_train = [features[idx] for idx in train_ids]
    fold_y_train = np.asarray([labels[idx] for idx in train_ids])
    fold_X_test = [features[idx] for idx in test_ids]
    fold_y_test = np.asarray([labels[idx] for idx in test_ids])
    
    #uso dict_vectorizer per trasformare fold_X_train e fold_X_test in matrici numeriche (verranno sparse)
    fold_X_train = vectorizer.fit_transform(fold_X_train)
    fold_X_test = vectorizer.transform(fold_X_test)
    
    #creo e addestro un svc sul training dell'attuale fold
    kfold_svc = LinearSVC(dual=False)
    kfold_svc.fit(fold_X_train, fold_y_train)
    #faccio una predizione sul test dell'attuale fold
    fold_y_pred = kfold_svc.predict(fold_X_test)
    
    #calcolo l'accuratezza dell'svc nel fold
    fold_accuracy = accuracy_score(fold_y_test, fold_y_pred)
    
    #calcolo l'accuratezza nel fold anche di un dummy classifier con strategia most_frequent, per avere una baseline con cui confrontare l'accuratezza dell'svc
    dummy_clf = DummyClassifier(strategy="most_frequent")   # dummy classifier viene utilizzato per avere una baseline
    dummy_clf.fit(fold_X_train, fold_y_train)
    dummy_score = dummy_clf.score(fold_X_test, fold_y_test)
    
    #aggiungo le etichette vere e le etichette predette alle liste create inizialmente, per poter calcolare poi le metriche
    all_y_true += fold_y_test.tolist()
    all_y_pred += fold_y_pred.tolist()
    
    #stampo l'accuracy e la confronto con la baseline (l'accuracy del dummy classifier)
    print(f"Accuracy fold {i+1}: {fold_accuracy}, baseline: {dummy_score}")
    
print(classification_report(all_y_true, all_y_pred, zero_division=0))

5469 1368
5469 1368
5470 1367
5470 1367
5470 1367
Accuracy fold 1: 0.6827485380116959, baseline: 0.5986842105263158
Accuracy fold 2: 0.6637426900584795, baseline: 0.591374269005848
Accuracy fold 3: 0.6583760058522312, baseline: 0.60424286759327
Accuracy fold 4: 0.6561814191660571, baseline: 0.5874177029992684
Accuracy fold 5: 0.6561814191660571, baseline: 0.5954645208485735
              precision    recall  f1-score   support

           0       0.65      0.92      0.76      4071
           1       0.71      0.29      0.41      2766

    accuracy                           0.66      6837
   macro avg       0.68      0.60      0.59      6837
weighted avg       0.68      0.66      0.62      6837



Delle 4 rappresentazioni prese in considerazione, quella basata su bigrammi di caratteri ha portato a prestazioni migliori. La proviamo dunque sui test set ufficiali del task.

In [24]:
#directory dei due test set
conllu_dir_test_1 = 'profiling_output_test_1/11936/'
conllu_dir_test_2 = 'profiling_output_test_2/11937/'

#estraggo i test set e li metto in due liste

all_documents_test_1 = [] #inizializza la lista dei documenti, per ora vuota
all_documents_test_2 = [] #inizializza la lista dei documenti, per ora vuota

#riempio la prima lista
for file_name in os.listdir(conllu_dir_test_1): #itera i file nella cartella
    file_path = os.path.join(conllu_dir_test_1, file_name)
    #per ogni file, crea un oggetto Document
    document = Document(file_path)
    #utilizza la funzione load_document_sentences per caricare il documento leggendone frasi e token
    load_document_sentences(document)
    #aggiunge il documento alla lista dei documenti
    all_documents_test_1.append(document)
    
#riempio la seconda lista
for file_name in os.listdir(conllu_dir_test_2): #itera i file nella cartella
    file_path = os.path.join(conllu_dir_test_2, file_name)
    #per ogni file, crea un oggetto Document
    document = Document(file_path)
    #utilizza la funzione load_document_sentences per caricare il documento leggendone frasi e token
    load_document_sentences(document)
    #aggiunge il documento alla lista dei documenti
    all_documents_test_2.append(document)

In [25]:
#addestro un svc lineare sul mio sistema migliore in modo da poterlo testare sui test set ufficiali
from sklearn.svm import LinearSVC

extract_documents_ngrams_normalized(all_documents, "char", 2)
features = [document.features for document in all_documents]
labels = [document.hs for document in all_documents]
#inizializzo il vettorizzatore
vectorizer = DictVectorizer()

#trasformo le features in matrici
X = vectorizer.fit_transform(features) #fitta il minmaxscaler e applica subito la trasformazione
y = np.array(labels)

#modello svc lineare
svc_model = LinearSVC(dual=False)
#addestro il modello
svc_model.fit(X, y)

LinearSVC(dual=False)

In [26]:
#estraggo le features dai documenti di test
extract_documents_ngrams_normalized(all_documents_test_1, "char", 2)
extract_documents_ngrams_normalized(all_documents_test_2, "char", 2)

#converto le features in una lista di dizionari, per entrambi i documenti di test
features_test_1 = [document.features for document in all_documents_test_1]
labels_test_1 = [document.hs for document in all_documents_test_1]
features_test_2 = [document.features for document in all_documents_test_2]
labels_test_2 = [document.hs for document in all_documents_test_2]

In [27]:
#non filtro le features del test set

print(f'Numero features test set 1: {get_num_features(features_test_1)}')
print(f'Numero features test set 2: {get_num_features(features_test_2)}')

Numero features test set 1: 737
Numero features test set 2: 1938


In [28]:
#converte in matrici numeriche; uso solo transform e non fit_transform perché così il vettorizzatore rimane fittato sui dati di training
X_test_1 = vectorizer.transform(features_test_1) 
X_test_2 = vectorizer.transform(features_test_2)

In [29]:
#predizione sul test1
y_pred_test_1 = svc_model.predict(X_test_1)
print(classification_report(labels_test_1, y_pred_test_1))

#predizione sul test2
y_pred_test_2 = svc_model.predict(X_test_2)
print(classification_report(labels_test_2, y_pred_test_2))

              precision    recall  f1-score   support

           0       0.69      0.83      0.75       319
           1       0.53      0.33      0.41       181

    accuracy                           0.65       500
   macro avg       0.61      0.58      0.58       500
weighted avg       0.63      0.65      0.63       500

              precision    recall  f1-score   support

           0       0.55      0.88      0.68       641
           1       0.68      0.27      0.39       622

    accuracy                           0.58      1263
   macro avg       0.62      0.57      0.53      1263
weighted avg       0.62      0.58      0.53      1263

